# Evaluate On Evaluation Set

## Summary

This notebook will do following steps:
 -  Pull validation data
 -  Load CTR predictions from CSV
 -  Load BaseBid Predictions for CSV
 -  Combine above using a formula
 -  Summarise Results

# Preparation

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from random import randint    
import seaborn as sns
from score_models import Scorer
import timeit
import random

%matplotlib inline

# Pull Validation Data

In [2]:
path = "C:/Users/Akis/OneDrive/Masters/Web Economics/Data/"
validation_df = pd.read_csv(path+'validation.csv')

In [3]:
avgCTR =validation_df.loc[validation_df["click"] == 1].shape[0]/(validation_df.shape[0])
print(avgCTR)

0.0006646376573167722


# Load CTR Predictions

Need a function that will combine multiple dataframes into one

In [4]:
path='C:/Users/Akis/OneDrive/Masters/Web Economics/Data/WebEcon Team Drive/predictions/'
teamACTRData = pd.read_csv(path+'lr_pCTR.csv')
teamACTR=teamACTRData[['click_proba']]
teamACTR = teamACTR.rename(index=str, columns={"click_proba": "pCTR"})

# Load Basebid Predictions

Need a function that will combine multiple dataframes into one

In [5]:
teamABaseBidData = pd.read_csv(path+'ridge_basebid.csv')
teamABaseBid=teamABaseBidData[['base_bid']]
print(teamABaseBid.head(2))


    base_bid
0  80.350004
1  95.867239


# Functions for combining pCTR and BaseBid

Need a function that will build a new dataframe containingbids of all teams by combining the previous two dataframes.

In [6]:
#given formula
def Function1(resBaseBid,resCTR):
    avCTR = 0.5
    Bids=resBaseBid.values * (resCTR/avCTR)
    #print("Bids:",Bids)
    return Bids

In [7]:
#exponential CTR
def Function2(reBaseBid,reCTR):
    avCTR = 0.5
    reCTR=reCTR.copy(True)
    reCTR=(((reCTR+0.5)**2)-0.5)
    reCTR[reCTR < 0] = 0
    Bids=reBaseBid.values * (reCTR/avCTR)
    return Bids

In [8]:
#Zero bid for unsure values and high bidding on expected clicks
def Function3(rBaseBid,rCTR):
    avCTR = 0.5
    rCTR=rCTR.copy(True)
    rCTR[rCTR < 0.5] = 0
    #rCTR=(((rCTR+0.5)**4)-0.5)
    Bids=rBaseBid.values*1.2 * (rCTR/avCTR)
    return Bids

In [9]:
def Function4(BaseBid,CTR):
    Bids = BaseBid*0+301
    #Bids = 301
    return Bids

In [10]:
def Function5(BaseBid,CTR):
    Bids =  pd.DataFrame(np.random.randint(295, 301+1, BaseBid.shape[0]))
    #Bids = 301
    return Bids

# Combine pCTR and BaseBid to create bids

In [11]:
Bids=pd.DataFrame(index=range(validation_df.shape[0]))
Bids['click']=validation_df['click']
Bids['payprice']=validation_df['payprice']

In [12]:
Bids['TeamAFormula1'] = Function1(teamABaseBid,teamACTR).values
Bids['TeamAFormula2'] = Function2(teamABaseBid,teamACTR).values

In [15]:
def EvaluateMultiple(bids,names,budget):
    #print(function(BaseBidPred,CTRPred))
    for i in range(len(names)):
        start = timeit.timeit()
        print("processing",i+1,"/",len(names),"... \n")
        s = Scorer()
        df_summary,df = s.set_df(bids, ['payprice',names[i]],budget,10)
        print(names[i],"results are:\n",df_summary)
        end = timeit.timeit()
        print(end - start,"seconds","\n \n")
    results=1
    return results

In [16]:
EvaluateMultiple(Bids,['TeamAFormula1','TeamAFormula2'],62500)

processing 1 / 2 ... 

TeamAFormula1 results are:
      budget  click      team_name   win
0 -6.000000      6  TeamAFormula1  3422
1 -3.745871      0       payprice   891
-0.00043820097544511327 seconds 
 

processing 2 / 2 ... 

TeamAFormula2 results are:
      budget  click      team_name   win
0 -7.000000      6  TeamAFormula2  3870
1 -1.306083      0       payprice  1031
-0.0005374162906548463 seconds 
 

Total time was: -0.0013428279728984194 seconds
